In [1]:
import numpy as np
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import fits
import plotly.graph_objects as go
from dustmaps.edenhofer2023 import Edenhofer2023Query

plot_save_directory = "./savedplots/"

In [2]:
def cylinder(r, h, nt=100, nv=50):
    """
    Parametrize cylinder of radius r, height h
    """
    theta = np.linspace(0, 2*np.pi, nt)
    v = np.linspace(-h/2, h/2, nv)
    theta, v = np.meshgrid(theta, v)

    y = r*np.cos(theta)
    z = r*np.sin(theta)
    x = v
    return x, y, z

def filledcylinder(r, h, nt=100, nh=50, nr=50):
    """
    Parametrize cylinder of radius r, height h
    """
    theta = np.linspace(0, 2*np.pi, nt)
    v = np.linspace(-h/2, h/2, nh)
    rr = np.linspace(0, r, nr)
    rr, theta, v = np.meshgrid(rr, theta, v)

    y = rr*np.cos(theta)
    z = rr*np.sin(theta)
    x = v
    return x, y, z

def rotationMatrix(a=0, b=0, c=0):
    """
    Return the Rotation Matrix with yaw a about z, pitch b about y, and roll c about x
    """
    Ra = np.array([
        [np.cos(a), -np.sin(a), 0],
        [np.sin(a), np.cos(a), 0],
        [0, 0, 1]])

    Rb = np.array([
        [np.cos(-b), 0, np.sin(-b)],
        [0, 1, 0],
        [-np.sin(-b), 0, np.cos(-b)]
    ])

    Rc = np.array([
        [1, 0, 0],
        [0, np.cos(c), -np.sin(c)],
        [0, np.sin(c), np.cos(c)]
    ])

    return Ra@Rb@Rc

___
## check_inside function

In [3]:
# write a function to check whether a point is inside the cylinder
def check_inside(x, y, z, r, h, t=(0,0,0), RotMat=np.array([[1,0,0],[0,1,0],[0,0,1]])):
    """
    Check if a point is inside the cylinder
    """
    # substraction of the translation
    x -= t[0]
    y -= t[1]
    z -= t[2]
    # rotation with the inverse of the rotation matrix
    x, y, z = np.linalg.inv(RotMat)@np.array([x, y, z])

    return z**2 + y**2 <= r**2 and -h/2 <= x <= h/2

___
## Old shit

In [4]:
#Cloud 1
R = 80
H = 300

nt = 90
nh = 150
nr = 70
x1, y1, z1 = filledcylinder(r=R, h=H, nt=nt, nh=nh, nr=nr)
#Get rotation Matrix
RotM = rotationMatrix(a=-(1/4)*np.pi, b=0, c=0)
#Define Translation Vector
t=(740, 40, 100)

v_f_min = 1e-6
v_f_max = 1.6e-3

In [5]:
#Stack coordinates for rotation
S = np.stack((x1, y1, z1), axis=2)

In [6]:
#Rotate by multiplying with rotMatrix
rotatedStack = RotM@S

#Separate again, and translate
xx = rotatedStack[:,:,0,:]+t[0]
yy = rotatedStack[:,:,1,:]+t[1]
zz = rotatedStack[:,:,2,:]+t[2]

In [7]:
#Visualize what the shell of the cylinder looks like:
cyl = go.Surface(x = xx[:,-1,:], y = yy[:,-1,:], z = zz[:,-1,:],
    opacity = 0.3,
    colorscale = [[0, "green"], [1, "green"]],
    showscale = False,
    name = "surface -1",
    showlegend = True
)

center = go.Scatter3d(
    x=[t[0]],
    y=[t[1]],
    z=[t[2]],
    mode = "markers",
    marker = dict(
        size=3, 
        color="yellow"
    ),
    name="center")

In [8]:
# Define skycoords using the coordinates of the points in the cylinder:
sc_cylinder = SkyCoord(
    xx.flatten() * u.pc,
    yy.flatten() * u.pc,
    zz.flatten() * u.pc,
    frame = 'galactic',
    representation_type = 'cartesian'
)

sc_cylinder.representation_type ="spherical"

#Definde SkyCoords in a orthogonal grid:
xt = np.linspace(np.min(xx), np.max(xx), 20)
yt = np.linspace(np.min(yy), np.max(yy), 20)
zt = np.linspace(np.min(zz), np.max(zz), 20)

#xt = np.arange(-310, -30, 10)
#yt = np.arange(-690, -420, 10)
#zt = np.arange(-80, 80, 10)

xt_grid, yt_grid, zt_grid = np.meshgrid(xt, yt, zt)

sc_grid = SkyCoord(
    xt_grid.flatten() * u.pc,
    yt_grid.flatten() * u.pc,
    zt_grid.flatten() * u.pc,
    frame = 'galactic',
    representation_type = 'cartesian'
)

sc_grid.representation_type = "spherical"

In [9]:
#Query Edenhofer2023:
dust_grid = Edenhofer2023Query().query(sc_grid)

Optimizing map for querying (this might take a couple of seconds)...


In [10]:
x_range = (np.min((np.min(xx), np.min(xt))), np.max((np.max(xx), np.max(xt))))
y_range = (np.min((np.min(yy), np.min(yt))), np.max((np.max(yy), np.max(yt))))
z_range = (np.min((np.min(zz), np.min(zt))), np.max((np.max(zz), np.max(zt))))

yscale = (np.max(y_range)-np.min(y_range))/(np.max(z_range)-np.min(z_range))
xscale = (np.max(x_range)-np.min(x_range))/(np.max(z_range)-np.min(z_range))

___

## This is where the magic happens

In [11]:
points_in_space = np.stack((xt_grid.flatten(), yt_grid.flatten(), zt_grid.flatten(), dust_grid), axis=1)
points_in_cylinder = np.stack((xt_grid.flatten(), yt_grid.flatten(), zt_grid.flatten(), np.zeros_like(dust_grid)), axis=1)

In [12]:

for i in np.arange(0, dust_grid.shape[0]):
    if check_inside(points_in_space[i,0], points_in_space[i,1], points_in_space[i,2], R, H, t, RotM):
        points_in_cylinder[i,3] = points_in_space[i,3]

In [13]:
volume_cylinder = go.Volume(
    x = points_in_cylinder[:,0],
    y = points_in_cylinder[:,1],
    z = points_in_cylinder[:,2],
    value = points_in_cylinder[:,3],
    opacity = 0.3,
    isomin = 2e-4,
    isomax = 5e-3,
    surface_count = 10
)

volume_cylinder_flat = go.Volume(
    x = points_in_cylinder[:,0],
    y = points_in_cylinder[:,1],
    z = points_in_cylinder[:,2],
    value = points_in_cylinder[:,3],
    flatshading = True,
    opacityscale = [[0,0], [1, 1]],
    isomin = v_f_min,
    isomax = v_f_max,
    surface = dict(show=True, count=1)
)

In [15]:
#Visualize the result
layout_3d = go.Layout(
    template = "plotly_white",
    #paper_bgcolor = "black",
    #plot_bgcolor = "black",
    title = "Dust",
    scene = dict(
        aspectmode = 'manual',
        aspectratio = dict(x = xscale, y = yscale, z = 1),
        xaxis = dict(title = "x", range = x_range),
        yaxis = dict(title = "y", range = y_range),
        zaxis = dict(title = "z", range = z_range),
    ),
    height=500,
    width=800
)

fig_3d_dust = go.Figure(data = (volume_cylinder_flat, cyl, center), layout = layout_3d)
fig_3d_dust.write_html(plot_save_directory + "Cylindertest2.html")
fig_3d_dust.show()